Mari kita jelaskan cara kerja kode tersebut secara berurutan:

1. **Inisialisasi Basis Pengetahuan:**
   - `knowledge_base = KnowledgeBase()`: Membuat objek basis pengetahuan yang akan menyimpan aturan-aturan.

2. **Aturan Awal dalam Sistem:**
   - `initial_rule = Rule(condition=lambda x: x['temperature'] > 25, conclusion="Pakailah pakaian ringan.")`: Membuat aturan awal yang menyatakan jika suhu lebih dari 25 derajat Celsius, maka rekomendasi adalah "Pakailah pakaian ringan."
   - `knowledge_base.add_rule(initial_rule)`: Menambahkan aturan awal ke basis pengetahuan.

3. **Kasus Uji Awal:**
   - `initial_test_data = {'temperature': 30, 'weather': 'cerah'}`: Membuat data uji awal dengan suhu 30 derajat Celsius dan cuaca cerah.

4. **Wawancara dengan Ahli:**
   - `expert_validator = ExpertValidator(knowledge_base, Expert())`: Membuat objek validator ahli dengan menggunakan basis pengetahuan dan objek ahli.
   - `expert_validator.expert_interview()`: Melakukan wawancara dengan ahli dan menampilkan umpan balik ahli.

5. **Validasi Berkelanjutan dengan Pembaruan Pengetahuan:**
   - `updates = [...]`: Menyiapkan pembaruan pengetahuan dalam bentuk aturan dan data uji.
   - `expert_validator.continuous_validation(updates)`: Melakukan validasi berkelanjutan dengan menambahkan aturan baru dan mengevaluasi hasil sistem setelah setiap pembaruan.

6. **Kasus Uji untuk Evaluasi Kinerja:**
   - `performance_evaluator = PerformanceEvaluator(knowledge_base)`: Membuat objek evaluator kinerja dengan menggunakan basis pengetahuan.
   - `test_cases = [...]`: Menyiapkan kasus uji untuk evaluasi kinerja.

7. **Evaluasi Kinerja:**
   - `performance_evaluator.evaluate_performance(test_cases)`: Melakukan evaluasi kinerja sistem dengan menggunakan kasus uji dan menampilkan hasilnya.

8. **Manajemen Konflik:**
   - Dalam kelas `KnowledgeBase`, ada implementasi sederhana untuk manajemen konflik. Jika terdeteksi konflik antara aturan yang akan ditambahkan dengan aturan yang sudah ada, aturan baru tidak akan ditambahkan ke basis pengetahuan.

Setiap langkah dalam kode tersebut dilakukan untuk menggambarkan berbagai aspek Knowledge Validation, seperti konsistensi pengetahuan, validasi terhadap pengetahuan ahli, uji kinerja sistem, dan manajemen konflik. Program ini menciptakan dan mengelola basis pengetahuan yang berkembang seiring waktu dan menguji sistem untuk memastikan konsistensi dan akurasi.

# Buat basis

In [22]:
class KnowledgeBase:
    def __init__(self):
        self.rules = []

    def add_rule(self, rule):
        if not self.has_conflict(rule):
            self.rules.append(rule)
        else:
            print(f"Konflik Ditemukan. Aturan tidak ditambahkan: {rule.condition.__name__}")

    def has_conflict(self, new_rule):
        for existing_rule in self.rules:
            if self.detect_conflict(existing_rule, new_rule):
                return True
        return False

    def detect_conflict(self, rule1, rule2):
        return rule1.condition.__name__ == rule2.condition.__name__

    def evaluate(self, input_data):
        for rule in self.rules:
            if rule.evaluate(input_data):
                return rule.conclusion
        return "No conclusion"



In [23]:
class Rule:
    def __init__(self, condition, conclusion):

        self.condition = condition
        self.conclusion = conclusion

    def evaluate(self, input_data):
        return self.condition(input_data)


# Masukan dari Pakar(Expert)

In [24]:
class Expert:
    def provide_feedback(self):
        return "Pengetahuan ini mencerminkan dengan baik kondisi di lapangan."

# Evaluasi

In [25]:
class PerformanceEvaluator:
    def __init__(self, knowledge_base):
        self.knowledge_base = knowledge_base

    def evaluate_performance(self, test_cases):
        correct_predictions = 0
        total_cases = len(test_cases)

        for test_case in test_cases:
            input_data = test_case['input']
            expected_output = test_case['output']

            result = self.knowledge_base.evaluate(input_data)

            if result == expected_output:
                correct_predictions += 1

            print(f"Data Uji: {input_data}")
            print(f"Hasil yang Diharapkan: {expected_output}")
            print(f"Hasil Sistem: {result}")
            print("--------------------")

        accuracy = (correct_predictions / total_cases) * 100
        print(f"Akurasi Sistem: {accuracy}%")

In [26]:
class ExpertValidator:
    def __init__(self, knowledge_base, expert):
        self.knowledge_base = knowledge_base
        self.expert = expert

    def expert_interview(self):
        expert_feedback = self.expert.provide_feedback()
        print("Umpan Balik Ahli:")
        print(expert_feedback)
        print("--------------------")
        print("  ")

    def continuous_validation(self, updates):
        for update in updates:
            self.knowledge_base.add_rule(update['rule'])

            result = self.knowledge_base.evaluate(update['test_data'])

            print(f"Data Uji setelah Pembaruan: {update['test_data']}")
            print(f"Hasil Sistem setelah Pembaruan: {result}")
            print("--------------------")

In [27]:
# Inisialisasi basis pengetahuan
knowledge_base = KnowledgeBase()

# Aturan awal dalam sistem
initial_rule = Rule(condition=lambda x: x['temperature'] > 25, conclusion="Pakailah pakaian ringan.")
knowledge_base.add_rule(initial_rule)

# Kasus uji awal
initial_test_data = {'temperature': 30, 'weather': 'cerah'}

# Membuat objek ExpertValidator
expert_validator = ExpertValidator(knowledge_base, Expert())

# Melakukan wawancara dengan ahli
expert_validator.expert_interview()

# Melakukan validasi berkelanjutan dengan pembaruan pengetahuan
updates = [
    {'rule': Rule(condition=lambda x: x['weather'] == 'hujan', conclusion="Bawa payung."),
     'test_data': {'temperature': 20, 'weather': 'hujan'}}
]

expert_validator.continuous_validation(updates)

# Kasus uji untuk evaluasi kinerja
performance_evaluator = PerformanceEvaluator(knowledge_base)
test_cases = [
    {'input': {'temperature': 30, 'weather': 'cerah'}, 'output': 'Pakailah pakaian ringan.'},
    {'input': {'temperature': 10, 'weather': 'hujan'}, 'output': 'Bawa payung.'},
    {'input': {'temperature': 20, 'weather': 'cerah'}, 'output': 'Pakailah pakaian ringan.'},
    {'input': {'temperature': 5, 'weather': 'cerah'}, 'output': 'No conclusion'},  # Tidak ada aturan yang cocok
]

# Melakukan evaluasi kinerja
performance_evaluator.evaluate_performance(test_cases)


Umpan Balik Ahli:
Pengetahuan ini mencerminkan dengan baik kondisi di lapangan.
--------------------
Konflik Ditemukan. Aturan tidak ditambahkan: <lambda>
Data Uji setelah Pembaruan: {'temperature': 20, 'weather': 'hujan'}
Hasil Sistem setelah Pembaruan: No conclusion
--------------------
Data Uji: {'temperature': 30, 'weather': 'cerah'}
Hasil yang Diharapkan: Pakailah pakaian ringan.
Hasil Sistem: Pakailah pakaian ringan.
--------------------
Data Uji: {'temperature': 10, 'weather': 'hujan'}
Hasil yang Diharapkan: Bawa payung.
Hasil Sistem: No conclusion
--------------------
Data Uji: {'temperature': 20, 'weather': 'cerah'}
Hasil yang Diharapkan: Pakailah pakaian ringan.
Hasil Sistem: No conclusion
--------------------
Data Uji: {'temperature': 5, 'weather': 'cerah'}
Hasil yang Diharapkan: No conclusion
Hasil Sistem: No conclusion
--------------------
Akurasi Sistem: 50.0%
